## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Download

Retrieve the page

In [4]:

#url = "https://www.placer.ca.gov/DocumentCenter/View/46267/dashboard"

In [6]:
url = "https://itwebservices.placer.ca.gov/coviddashboard"

In [7]:
page = requests.get(url)

## Parse

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Find script tag with the Infogram data

In [10]:
for elem in soup(text=re.compile(r"Foresthill")):
    script_content = elem

In [11]:
data = json.loads(script_content)

In [12]:
zip_cities_list = data["x"]["calls"][1]["args"][-3]

In [15]:
dict_list = []

In [16]:
TAG_RE = re.compile(r"<[^>]+>")

In [17]:
for z in zip_cities_list:
    split_zips = z.split("<br/>")
    area = TAG_RE.sub("", str(split_zips[0]))
    zip_code, city = area.split(" - ")
    clean_cases = re.sub("Number of cases: ".lower(), "", str(split_zips[1]).lower())
    d = dict(area=area, city=city, zip_code=zip_code, confirmed_cases=clean_cases)
    dict_list.append(d)

In [18]:
df = pd.DataFrame(dict_list)

Get timestamp

In [20]:
time_div = soup.find("div", id="dashboard-data-last-updated")

In [21]:
timestamp = time_div.find("span", class_="value-output").get_text()

In [22]:
latest_date = pd.to_datetime(timestamp).date()

In [23]:
df["county_date"] = latest_date

In [24]:
df.insert(0, "county", "Placer")

Clean up

In [25]:
df["area"] = df["area"].str.replace(" - ", ": ")

In [26]:
df = df[["county", "area", "confirmed_cases", "county_date", "zip_code"]].rename(
    columns={"zip_code": "zip"}
)

## Vet

In [27]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("Placer County's scraper is missing rows")

In [28]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("Placer County's scraper has more rows than before")

## Export

Set date

In [29]:
tz = pytz.timezone("America/Los_Angeles")

In [30]:
today = datetime.now(tz).date()

In [31]:
slug = "placer"

In [33]:
df

,county,area,confirmed_cases,county_date,zip
0,Placer,96162: Truckee,<6,2021-12-22,96162
1,Placer,95658: Newcastle,571,2021-12-22,95658
2,Placer,95714: Dutch Flat,25,2021-12-22,95714
3,Placer,95715: Emigrant Gap,13,2021-12-22,95715
4,Placer,96148: Tahoe Vista,111,2021-12-22,96148
5,Placer,95661: Roseville,3686,2021-12-22,95661
6,Placer,96161: Truckee,178,2021-12-22,96161
7,Placer,95650: Loomis,1290,2021-12-22,95650
8,Placer,96146: Olympic Valley,135,2021-12-22,96146
9,Placer,95678: Roseville,4975,2021-12-22,95678


In [34]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [35]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [36]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [37]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [38]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)